# Stage 5 (3.4 recs_for_cold_users)


# Импортируем библиотеки

In [1]:
import warnings

from datetime import datetime, timedelta

import dill

import numpy as np
import pandas as pd
import polars as pl

from rectools import Columns
from rectools.dataset import Dataset as RTDataset

from rectools.models import (
    PopularModel,
)
from mab2rec import BanditRecommender, LearningPolicy


from tqdm.auto import tqdm

from popular_bandit import PopularBanditRecommender
from metrics import RecommenderMetrics


warnings.filterwarnings("ignore")

RANDOM_STATE = 42

## Data paths

In [2]:
data_path = "../data/closed/"

In [3]:
models_path = "../data/models/"

In [4]:
candidates_data_path = models_path + "candidates_data/"

# Load Data & create Rectools Dataset

In [5]:
# Холодные пользователи из test_df, которым будут выданы
# рекомондации с помощью bandit recommender
with open(data_path + "test_only_users.dill", "rb") as f:
    test_only_users = dill.load(f)

test_only_users, test_only_users.shape

(array([4194307, 3145732, 4194309, ..., 1048568, 2097148, 4194301]), (542353,))

In [6]:
def create_rectools_dataset(models_data: pl.LazyFrame) -> RTDataset:
    """
    Create rectools dataset

    Args:
        models_A Polars LazyFrame containing interaction data with columns
                    'user_id', 'item_id', 'dt', and 'cum_weight'.

    Returns:
        A rectools Dataset object.
    """
    return RTDataset.construct(
        # Изменим датасетпод стандарт `rectools`
        # Оставим только нужные колонки и переименуем
        interactions_df=models_data.select(
            [
                "user_id",
                "item_id",
                "dt",
                "cum_weight",
            ]
        )
        .rename(
            {
                "user_id": Columns.User,
                "item_id": Columns.Item,
                "dt": Columns.Datetime,
                "cum_weight": Columns.Weight,
            }
        )
        .collect()
        # преобразуем в формат pandas
        .to_pandas(),
    )

In [7]:
current_dataset = (
    create_rectools_dataset(
        models_data=pl.concat(
            [
                pl.scan_parquet(data_path + "base_models_data.parquet").select(
                    [
                        "user_id",
                        "item_id",
                        "dt",
                        "cum_weight",
                    ]
                ),
                pl.scan_parquet(data_path + "ranker_data.parquet").select(
                    [
                        "user_id",
                        "item_id",
                        "dt",
                        "cum_weight",
                    ]
                ),
            ],
            how="vertical",
        )
    )
)

13 s

# Rectools PopularModel

In [9]:
# Train popular model on newer data
popular_model = PopularModel()

popular_model.fit(current_dataset)

# Save model
with open(models_path + "popular_model.dill", "wb") as f:
    dill.dump(popular_model, f)

8 s

Для оптимизации процесса `BanditRecommender` будем обучать только на предметах входящих в `top_k=1000` от популярной модели.

> Так как планируется выдавать кандидатов для бандита из `top_k=50`, то `+2000%` к изначальному топу будет обеспечивать накопление популярности у возможных будущих кандидатов на попадание в руки бандиту

In [10]:
model_name = "pop"

In [11]:
pl.from_pandas(
    popular_model.recommend(
        users=[-1],
        dataset=current_dataset,
        # выдаем n_candidates кандидатов
        k=1000,
        # рекомендуем уже просмотренные товары
        filter_viewed=False,
    )
).rename(
    {
        "score": "pop_score",
        "rank": "pop_rank",
    }
).write_parquet(
    candidates_data_path + f"candidates_pop.parquet"
)

# BanditRecommender

In [13]:
mab_data = (
    pl.concat(
        [
            pl.scan_parquet(data_path + "base_models_data.parquet").select(
                [
                    "user_id",
                    "item_id",
                    "dt",
                    "ui_inter",
                    "u_total_inter",
                ]
            ),
            pl.scan_parquet(data_path + "ranker_data.parquet").select(
                [
                    "user_id",
                    "item_id",
                    "dt",
                    "ui_inter",
                    "u_total_inter",
                ]
            ),
        ],
        how="vertical",
    )
    .filter(
        (
            (
                pl.col("item_id").is_in(
                    pl.scan_parquet(candidates_data_path + f"candidates_pop.parquet")
                    .select("item_id")
                    .collect()
                    .to_numpy()
                    .flatten()
                )
            )
        )
    )
    .with_columns(
        (pl.col("ui_inter") > 2)
        .cast(pl.UInt8)
        .alias("binary_weight"),
    )
    .select(
        "user_id",
        "item_id",
        "dt",
        "binary_weight",
    )
)

In [ ]:
mab_model = BanditRecommender(
    LearningPolicy.ThompsonSampling(),
    top_k=15,
    n_jobs=-1,
)

min_date = mab_data.select("dt").min().collect().item()
max_date = mab_data.select("dt").max().collect().item()

left, right = min_date, min_date + timedelta(hours=4)
chunk = mab_data.filter(pl.col("dt").is_between(left, right, closed="left")).collect()

mab_model.fit(
        decisions=chunk["item_id"].to_numpy(),
        rewards=chunk["binary_weight"].to_numpy(),
    )

print(f"Fitted: {left}, {right}")


while right <= max_date:

    left, right = right, right + timedelta(hours=4)

    chunk = mab_data.filter(
        pl.col("dt").is_between(left, right, closed="left")
    ).collect()

    mab_model.partial_fit(
        decisions=chunk["item_id"].to_numpy(),
        rewards=chunk["binary_weight"].to_numpy(),
    )

    print(f"Fitted: {left}, {right}")

> Видимые накладки и несовпадения в выводе времени --- только результат реализации самого вывода, внутри все даты верные.

In [17]:
# Save model
with open(models_path + "mab_model.dill", "wb") as f:
    dill.dump(mab_model, f)

# PopularBanditRecommender

In [18]:
pop_bandit_model = PopularBanditRecommender(
    dataset=current_dataset,
    path_bandit_model=models_path + "mab_model.dill",
    path_popular_model=models_path + "popular_model.dill",
    top_k=15,
)

## Predictions

In [19]:
pl.from_pandas(
    pop_bandit_model.predict(
        test_only_users,
        pop_k=50,
        pre_gen_recs=True,
        pre_gen_n=100,
    )
).write_parquet(candidates_data_path + "candidates_mab.parquet")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

42 s

## Check metrics (even if predicitons are pretty random)

In [ ]:
predictions = (
    pl.scan_parquet(candidates_data_path + "candidates_mab.parquet")
    .filter(pl.col("mab_rank") <= 15)
    .select(["user_id", "item_id"])
    .group_by("user_id")
    .agg(pl.col("item_id").alias("mab_recs"))
    .collect()
)

In [ ]:
test_df = (
    pl.scan_parquet(data_path + "test_df.parquet")
    .filter(pl.col("user_id").is_in(test_only_users))
    .collect()
)

In [12]:
RecommenderMetrics.evaluate_recommender(
    test_df.join(
        other=predictions,
        how="left",
        on="user_id",
    ),
    model_preds_col="mab_recs",
)

{'ndcg@k': 0.014726990829989128,
 'recall@k': 0.006159225205314413,
 'map@k': 0.001800778284694752}

In [24]:
test_df = pd.merge(
    test_df[test_df["user_id"].isin(predictions["user_id"].unique())],
    predictions,
    how="left",
    on="user_id",
)

In [14]:
for k in [3, 5, 10, 15]:
    print(
        f"""k = {k}: {RecommenderMetrics.evaluate_recommender(
        test_df.join(
            other=predictions,
            how="left",
            on="user_id",
        ),
        model_preds_col="mab_recs",
        k=k,
        )}\n"""
    )

k = 3: {'ndcg@k': 0.002795271063609042, 'recall@k': 0.000998378434266507, 'map@k': 0.0016719840317202187}

k = 5: {'ndcg@k': 0.005516108307809027, 'recall@k': 0.0019671743415740693, 'map@k': 0.001551871608015853}

k = 10: {'ndcg@k': 0.014726990829989128, 'recall@k': 0.006159225205314413, 'map@k': 0.001800778284694752}

k = 15: {'ndcg@k': 0.01963840319000725, 'recall@k': 0.007917476260422574, 'map@k': 0.0018018104371959212}



k = 3: {'ndcg@k': 0.002795271063609042, 'recall@k': 0.000998378434266507, 'map@k': 0.0016719840317202187}

k = 5: {'ndcg@k': 0.005516108307809027, 'recall@k': 0.0019671743415740693, 'map@k': 0.001551871608015853}

k = 10: {'ndcg@k': 0.014726990829989128, 'recall@k': 0.006159225205314413, 'map@k': 0.001800778284694752}

k = 15: {'ndcg@k': 0.01963840319000725, 'recall@k': 0.007917476260422574, 'map@k': 0.0018018104371959212}